In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def create_cpu_plots(csv_filepath):
    """
    Δημιουργεί διαγράμματα χρόνου και επιτάχυνσης για την CPU από ένα CSV αρχείο,
    χρησιμοποιώντας μια προκαθορισμένη παλέτα χρωμάτων.
    """
    try:
        df = pd.read_csv(csv_filepath)
        df.columns = df.columns.str.strip()
    except FileNotFoundError:
        print(f"Σφάλμα: Το αρχείο '{csv_filepath}' δεν βρέθηκε.")
        return

    matrix_dimensions = sorted(df['SquareMatrixDimension'].unique())
    processes_counts = sorted(df['NumberOfProcesses'].unique())

    # --- Ορισμός της παλέτας χρωμάτων ---
    # Ένα χρώμα για κάθε διάσταση πίνακα
    color_palette = ['#f39c12', '#27ae60', '#2980b9', '#76448a']

    # --- Διάγραμμα 1: Χρόνος Εκτέλεσης ---

    fig1, ax1 = plt.subplots(figsize=(14, 8))
    min_time_for_plot = 1e-4

    # Χρησιμοποιούμε enumerate για να έχουμε έναν δείκτη (idx) για την παλέτα χρωμάτων
    for idx, dim in enumerate(matrix_dimensions):
        subset = df[df['SquareMatrixDimension'] == dim].copy()
        subset.loc[subset['ExecutionTime'] <= 0, 'ExecutionTime'] = min_time_for_plot

        # Ορίζουμε το χρώμα της γραμμής από την παλέτα μας
        line_color = color_palette[idx % len(color_palette)]

        ax1.plot(subset['NumberOfProcesses'], subset['ExecutionTime'],
                 marker='o', linestyle='-', label=f'Custom GEMM {dim}x{dim}', color=line_color)

        numpy_time = subset['NumpyExecutionTime'].iloc[0]
        plot_numpy_time = numpy_time if numpy_time > 0 else min_time_for_plot

        ax1.axhline(y=plot_numpy_time, color=line_color, linestyle='--',
                    linewidth=2, label=f'NumPy {dim}x{dim}')

    ax1.set_title('Απόδοση GEMM σε CPU: Χρόνος Εκτέλεσης vs Αριθμός Διεργασιών', fontsize=16)
    ax1.set_xlabel('Αριθμός Διεργασιών', fontsize=12)
    ax1.set_ylabel('Χρόνος Εκτέλεσης (δευτερόλεπτα) - Λογαριθμική Κλίμακα', fontsize=15)
    ax1.set_xticks(processes_counts)
    ax1.set_yscale('log')
    ax1.grid(which='major', linestyle=':', linewidth='0.7')

    ax1.legend(title='Υλοποίηση & Διάσταση', bbox_to_anchor=(1.03, 1), loc='upper left', frameon=True, fontsize=16)

    plt.savefig('execution_time_cpu.png', dpi=300, bbox_inches='tight')
    plt.close(fig1)
    print("Το γράφημα 'execution_time_cpu.png' αποθηκεύτηκε.")

    # --- Διάγραμμα 2: Επιτάχυνση (Speedup) ---

    fig2, ax2 = plt.subplots(figsize=(14, 8))

    for idx, dim in enumerate(matrix_dimensions):
        subset = df[df['SquareMatrixDimension'] == dim]

        # Ορίζουμε το χρώμα της γραμμής από την παλέτα μας
        line_color = color_palette[idx % len(color_palette)]

        ax2.plot(subset['NumberOfProcesses'], subset['speedupToSerial'],
                 marker='o', linestyle='-', label=f'Matrix {dim}x{dim}', color=line_color)

    ax2.plot(processes_counts, processes_counts, linestyle='--', color='k', alpha=0.7, label='Ιδανική Επιτάχυνση (y=x)')

    ax2.set_title('Κλιμακωσιμότητα GEMM σε CPU: Speedup vs Αριθμός Διεργασιών', fontsize=16)
    ax2.set_xlabel('Αριθμός Διεργασιών', fontsize=15)
    ax2.set_ylabel('Speedup (βελτίωση vs 1 διεργασία)', fontsize=15)
    ax2.set_xticks(processes_counts)
    ax2.set_ylim(bottom=0)
    ax2.grid(which='major', linestyle=':', linewidth='0.7')

    ax2.legend(title='Διάσταση Πίνακα', bbox_to_anchor=(1.03, 1), loc='upper left', frameon=True, fontsize=15)

    plt.savefig('speedup_cpu.png', dpi=300, bbox_inches='tight')
    plt.close(fig2)
    print("Το γράφημα 'speedup_cpu.png' αποθηκεύτηκε.")


if __name__ == '__main__':
    csv_file_path = 'MultiprocMatMulTests.csv'
    create_cpu_plots(csv_file_path)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def create_gpu_plots(csv_filepath):
    """
    Δημιουργεί stacked bar charts για την απόδοση της GPU, με προσαρμοσμένα χρώματα.
    """
    try:
        df = pd.read_csv(csv_filepath)
        df.columns = df.columns.str.strip()
    except FileNotFoundError:
        print(f"Σφάλμα: Το αρχείο '{csv_filepath}' δεν βρέθηκε.")
        return

    df['ComputationTime'] = df['TotalTime'] - df['TransfersTime']
    matrix_dimensions = sorted(df['SquareMatrixDimension'].unique())

    # --- Ορισμός χρωμάτων ---
    computation_color = '#76448a'
    transfer_color = '#2980b9'

    for dim in matrix_dimensions:
        print(f"Δημιουργία γραφήματος για διάσταση {dim}x{dim}...")
        subset = df[df['SquareMatrixDimension'] == dim].set_index('Version')

        fig, ax = plt.subplots(figsize=(12, 8))

        # --- Σχεδίαση με τα νέα χρώματα ---
        bars1 = ax.bar(subset.index, subset['TransfersTime'],
                       label='Χρόνος Μεταφορών (CPU-GPU)', color=transfer_color, zorder=3)

        bars2 = ax.bar(subset.index, subset['ComputationTime'],
                       bottom=subset['TransfersTime'], label='Χρόνος Υπολογισμού (Kernel)', color=computation_color, zorder=3)

        ax.set_title(f'Ανάλυση Χρόνου GEMM σε GPU (Πίνακας {dim}x{dim})', fontsize=16, pad=20)
        ax.set_ylabel('Χρόνος Εκτέλεσης (ms)', fontsize=15)
        ax.set_xlabel('Τύπος Υλοποίησης', fontsize=15)
        plt.xticks(rotation=0, ha='center')

        ax.legend(title='Κατανομή Χρόνου', bbox_to_anchor=(1.03, 1), loc='upper left', frameon=True, fontsize=15)
        ax.grid(which='major', axis='y', linestyle=':', linewidth='0.7', color='gray', zorder=0)
        ax.spines[['top', 'right']].set_visible(False)

        # Ετικέτες ΜΕΣΑ στις μπάρες
        for bar in bars1:
            height = bar.get_height()
            if height > subset['TotalTime'].max() * 0.005:
                ax.text(bar.get_x() + bar.get_width() / 2, height / 2,
                        f'{height:.2f}', ha='center', va='center', color='black', weight='bold') 

        for bar in bars2:
            height = bar.get_height()
            bottom = bar.get_y()
            if height > subset['TotalTime'].max() * 0.05:
                ax.text(bar.get_x() + bar.get_width() / 2, bottom + height / 2,
                        f'{height:.2f}', ha='center', va='center', color='white', weight='bold') 

        # Ετικέτες με το ΣΥΝΟΛΙΚΟ άθροισμα
        for index, row in subset.iterrows():
            total_time = row['TotalTime']
            ax.text(index, total_time, f'{total_time:.2f}', ha='center', va='bottom', fontsize=10,
                    bbox=dict(facecolor='white', alpha=0.6, boxstyle='round,pad=0.2', ec='none'))

        ax.set_ylim(top=subset['TotalTime'].max() * 1.15)

        output_filename = f'gpu_performance_{dim}x{dim}.png'
        plt.savefig(output_filename, dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"Το γράφημα '{output_filename}' αποθηκεύτηκε.")

if __name__ == '__main__':
    csv_file_path = 'CUDAMatMulTests.csv'
    create_gpu_plots(csv_file_path)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re 
import os

def parse_filename(filename):
    """
    Διαβάζει ένα όνομα αρχείου και εξάγει τις παραμέτρους της εκπαίδευσης.
    Παράδειγμα ονόματος: NN_epochs-1000_batchSize-128_outMLP0-128_outMLP1-64.csv
    """
    # Χρησιμοποιούμε regular expressions για να βρούμε τους αριθμούς
    match = re.search(
        r'NN_epochs-(\d+)_batchSize-(\d+)_outMLP0-(\d+)_outMLP1-(\d+)',
        filename
    )
    if match:
        epochs = match.group(1)
        batch_size = match.group(2)
        mlp0 = match.group(3)
        mlp1 = match.group(4)
        return epochs, batch_size, mlp0, mlp1
    return None, None, None, None


def create_nn_plot(csv_filepath):
    """
    Δημιουργεί ένα stacked bar chart για ένα συγκεκριμένο αρχείο CSV
    αποτελεσμάτων του νευρωνικού δικτύου.
    """
    try:
        df = pd.read_csv(csv_filepath)
        df.columns = df.columns.str.strip()
    except FileNotFoundError:
        print(f"Σφάλμα: Το αρχείο '{csv_filepath}' δεν βρέθηκε.")
        return

    # Εξαγωγή παραμέτρων από το όνομα του αρχείου
    filename = os.path.basename(csv_filepath)
    epochs, batch_size, mlp0, mlp1 = parse_filename(filename)

    # Μετονομασία στηλών για συνέπεια
    df.rename(columns={'TotalTime_s': 'TotalTime', 'TransfersTime_s': 'TransfersTime'}, inplace=True)
    df['ComputationTime'] = df['TotalTime'] - df['TransfersTime']
    df.set_index('Version', inplace=True)

    # --- Ορισμός χρωμάτων ---
    computation_color = '#148f77'
    transfer_color = '#f55dfc'

    # --- Δημιουργία Γραφήματος ---
    fig, ax = plt.subplots(figsize=(12, 8))

    bars_transfers = ax.bar(df.index, df['TransfersTime'],
                            label='Χρόνος Μεταφορών (CPU-GPU)', color=transfer_color, zorder=3)
    bars_computation = ax.bar(df.index, df['ComputationTime'],
                              bottom=df['TransfersTime'], label='Χρόνος Υπολογισμού (NN)', color=computation_color, zorder=3)

    # --- Δυναμικός Τίτλος και Ετικέτες ---
    title = f'Ανάλυση Χρόνου Εκπαίδευσης Νευρωνικού Δικτύου'
    subtitle = f'(Epochs: {epochs}, Batch Size: {batch_size}, Layers: {mlp0}/{mlp1})'
    ax.set_title(f"{title}\n{subtitle}", fontsize=16, pad=20)

    ax.set_ylabel('Συνολικός Χρόνος Εκτέλεσης (s)', fontsize=15)
    ax.set_xlabel('Υλοποίηση Πυρήνα GEMM', fontsize=15)
    plt.xticks(rotation=0, ha='center', fontsize=12)
    plt.yticks(fontsize=12)

    ax.legend(title='Κατανομή Χρόνου', bbox_to_anchor=(1.03, 1), loc='upper left', frameon=True, fontsize=15)
    ax.grid(which='major', axis='y', linestyle=':', linewidth='0.7', color='gray', zorder=0)
    ax.spines[['top', 'right']].set_visible(False)

    # --- Ετικέτες στις μπάρες ---
    for bar in bars_computation:
        height = bar.get_height()
        if height > df['TotalTime'].max() * 0.05:
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_y() + height / 2,
                    f'{height:.2f}', ha='center', va='center', color='white', weight='bold', fontsize=10)

    for bar in bars_transfers:
        height = bar.get_height()
        if height > df['TotalTime'].max() * 0.05:
            ax.text(bar.get_x() + bar.get_width() / 2, height / 2,
                    f'{height:.2f}', ha='center', va='center', color='black', weight='bold', fontsize=10)

    for index, row in df.iterrows():
        total_time = row['TotalTime']
        ax.text(index, total_time, f'{total_time:.2f}s', ha='center', va='bottom',
                bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.2', ec='none'))

    ax.set_ylim(top=df['TotalTime'].max() * 1.15)

    # Δυναμικό όνομα αρχείου εξόδου
    output_filename = f'nn_performance_{epochs}e_{batch_size}b.png'
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Το γράφημα '{output_filename}' αποθηκεύτηκε.")


if __name__ == '__main__':
    # Λίστα με τα αρχεία CSV που θέλουμε να επεξεργαστούμε
    csv_files = [
        'NN_epochs-1_batchSize-8192_outMLP0-8192_outMLP1-4096.csv',
        'NN_epochs-1000_batchSize-128_outMLP0-128_outMLP1-64.csv'
    ]

    for file in csv_files:
        create_nn_plot(file)